 <div style="background-image: url('https://school.brainhackmtl.org/img/brainhackSchool.jpg');  background-size: 100% 100%;"> 



<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# Containers are not magic
<br/>

## BrainHack School 2020
<br/>

### Tristan Glatard
### 26 May, 2020

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>


# Week 1 strikes back

In Week 1, you learned why and how to use Docker and Singularity containers during [Peer's wonderful lecture](https://github.com/neurodatascience/course-materials-2020/blob/master/lectures/12-may/02-intro-to-containerization/neurodatascience_virtualization_2020.pdf). Now you can't work without containers, they're really magic! 

![CommitStrip](https://www.commitstrip.com/wp-content/uploads/2017/02/Strip-Ou-sont-les-tests-unitaires-english650-final.jpg "It's magic")
(only the last box of this is actually relevant, the first ones are a good intro to tomorrow's lecture)


# Goals

* Get to know more about the system (file system and processes) <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Tux.svg/440px-Tux.svg.png" alt="Tux" style="width: 100px;"/>
* Understand that containers aren't magic

# Method

* We will build "mocker", a tiny container engine

# Resources

![How containers work](https://jvns.ca/images/containers-cover.jpg "https://wizardzines.com/zines/containers")
(available at https://wizardzines.com/zines/containers)
* ["What even is a container?"](https://jvns.ca/blog/2016/10/10/what-even-is-a-container),  Julia Evans
* `man unshare`, `man cgroups`

# __mocker v0.1__: replacing file systems


## File system mounts

A file system provides a usable interface to storage devices, organized in files and directories. In Linux, file systems need to be appended, a.k.a _mounted_, under the `/` directory structure to be accessible. The location at which a file system is mounted is called a _mount point_. Mount points can be listed using command `df`:

In [1]:
!df -hT

Filesystem              Type      Size  Used Avail Use% Mounted on
devtmpfs                devtmpfs  7.7G     0  7.7G   0% /dev
tmpfs                   tmpfs     7.7G  276M  7.5G   4% /dev/shm
tmpfs                   tmpfs     7.7G  2.0M  7.7G   1% /run
tmpfs                   tmpfs     7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/mapper/fedora-root ext4       49G   31G   17G  66% /
tmpfs                   tmpfs     7.7G   10M  7.7G   1% /tmp
/dev/nvme0n1p2          ext4      976M  209M  700M  23% /boot
/dev/loop2              squashfs  161M  161M     0 100% /var/lib/snapd/snap/gnome-3-28-1804/116
/dev/loop3              squashfs   28M   28M     0 100% /var/lib/snapd/snap/snapd/6953
/dev/loop0              squashfs  227M  227M     0 100% /var/lib/snapd/snap/wine-platform-runtime/136
/dev/loop1              squashfs   55M   55M     0 100% /var/lib/snapd/snap/core18/1705
/dev/loop4              squashfs  227M  227M     0 100% /var/lib/snapd/snap/wine-platform-runtime/123
/dev/nvme0n1p1     

## File system types

There are different types of file systems, such as:
* tmpfs
* ext4, vfat
* squashfs
* nfs

OK, this is not _really_ relevant to containers but I thought you might be interested :)

Some directories in the file system are particularly important:
* /bin: contains many useful programs such as `ls`, `cd`, etc
* /etc: contains configuration files
* /proc: contains information about processes, mounts, etc

## chroot: changing the root file system

chroot is a system call to change the root of the current file system from `/` to a custom directory. It gives the illusion that a complete new file hierarchy was deployed, in isolation from the initial environment.

In [12]:
# chroot illustration
!wget bit.ly/fish-container -O fish.tar # download a tar archive file containing a full directory hierarchy
!mkdir -p container-root && (cd container-root && tar xf ../fish.tar) # unextract archive in directory container-root


--2020-05-24 20:34:56--  http://bit.ly/fish-container
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.github.com/jvns/1e8b2dcc2c9afac04acc4701112e3512/raw/5ef7f88cd089a70605b301cdd43f388d728f6817/fish.tar [following]
--2020-05-24 20:34:56--  https://gist.github.com/jvns/1e8b2dcc2c9afac04acc4701112e3512/raw/5ef7f88cd089a70605b301cdd43f388d728f6817/fish.tar
Resolving gist.github.com (gist.github.com)... 140.82.112.4
Connecting to gist.github.com (gist.github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/jvns/1e8b2dcc2c9afac04acc4701112e3512/raw/5ef7f88cd089a70605b301cdd43f388d728f6817/fish.tar [following]
--2020-05-24 20:34:56--  https://gist.githubusercontent.com/jvns/1e8b2dcc2c9afac04acc4701112e3512/raw/5ef7f88cd089a70605b301cdd43f388d

In [ ]:
!sudo chroot $PWD/container-root /bin/sh


[sudo] password for glatard: 

It looks like we're running on a different computer! Changing the root directory is a key concept of containers. 

# __mocker v0.2__: isolating processes

There are in fact many indices showing that the "chrooted" environment is still running on the "host" computer. To start with, we can still see and interact with other programs, or _processes_ on the host.

## Processes
A process is a running program. It has a number (ID) and is often associated with a command. Current processes can be listed with command `ps`:

In [4]:
!ps aux

USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root         1  0.0  0.0 170640 13168 ?        Ss   May23   0:22 /usr/lib/system
root         2  0.0  0.0      0     0 ?        S    May23   0:00 [kthreadd]
root         3  0.0  0.0      0     0 ?        I<   May23   0:00 [rcu_gp]
root         4  0.0  0.0      0     0 ?        I<   May23   0:00 [rcu_par_gp]
root         6  0.0  0.0      0     0 ?        I<   May23   0:00 [kworker/0:0H-e
root         9  0.0  0.0      0     0 ?        I<   May23   0:00 [mm_percpu_wq]
root        10  0.0  0.0      0     0 ?        S    May23   0:05 [ksoftirqd/0]
root        11  0.0  0.0      0     0 ?        I    May23   1:30 [rcu_sched]
root        12  0.0  0.0      0     0 ?        S    May23   0:00 [migration/0]
root        13  0.0  0.0      0     0 ?        S    May23   0:00 [cpuhp/0]
root        14  0.0  0.0      0     0 ?        S    May23   0:00 [cpuhp/1]
root        15  0.0  0.0      0     0 ?        S    May23   0:00 [migrat

`top` or `htop` are other useful commands to list proceses.

## Process trees

All processes but one have a parent, the program from which the process was launched, and processes might create child processes. The process tree can be viewed using `pstree` or `htop -t`:

In [6]:
!pstree

systemd─┬─ModemManager───2*[{ModemManager}]
        ├─NetworkManager─┬─dhclient
        │                └─2*[{NetworkManager}]
        ├─abrt-dbus───2*[{abrt-dbus}]
        ├─2*[abrt-dump-journ]
        ├─abrtd───2*[{abrtd}]
        ├─accounts-daemon───2*[{accounts-daemon}]
        ├─alsactl
        ├─atd
        ├─auditd───{auditd}
        ├─avahi-daemon───avahi-daemon
        ├─bluetoothd
        ├─boltd───2*[{boltd}]
        ├─chronyd
        ├─colord───2*[{colord}]
        ├─containerd───14*[{containerd}]
        ├─crond
        ├─cupsd
        ├─dbus-broker-lau───dbus-broker
        ├─dnsmasq───dnsmasq
        ├─dockerd───12*[{dockerd}]
        ├─firefox─┬─2*[Web Content───26*[{Web Content}]]
        │         ├─2*[Web Content───25*[{Web Content}]]
        │         ├─3*[Web Content───24*[{Web Content}]]
        │         ├─Web Content───27*[{Web Content}]
        │         ├─WebExtensions───25*[{WebExtensions}]
        │         ├─file:// Content───24*[{file:// Content}]
       

The `kill` command can be used to send signals to processes, using their PID. The default signal sent by `kill` is `TERM`, which terminates the process.

[demo in terminal]

Using these commands from the chrooted environment, it's clear that __mocker v0.1__ does not provide a complete illusion that we're on a different computer.

## Isolating processes through namespaces

Linux namespaces provide a way to isolate processes (and more) from the rest of the host. Namespaces are a key feature to enable containers. The `unshare` command can be used to create namespaces.

In [3]:
!sudo unshare -fp chroot $PWD/container-root /bin/sh -c "/bin/mount -t proc proc /proc && /bin/sh"

[sudo] password for glatard: 
[sudo] password for glatard: 

In [ ]:
nsenter (docker exec?)

In [ ]:
## Changing hostnames



# __mocker v0.3__: adding memory restrictions with cgroups

Maybe you don't want your memory-hungry containerized application to bloat your computer. Linux _cgroups_ (control groups) can allocate restricted amounts of memory (but also CPU and other resources) to a group of processes.

For instance, using cgroups, we can make sure that our container will only be allowed to use 10MB of memory:


In [ ]:
!sudo cgcreate -g "cpu,cpuacct,memory:mygroup"
!sudo cgset -r memory.limit_in_bytes=10000000 mygroup # 10 MB


[sudo] password for glatard: 
[sudo] password for glatard: [sudo] password for glatard: 

# Conclusion
Let's play some [flashcards](https://flashcards.wizardzines.com/container-basics)


# Future work

One day I'll finish this lecture and also talk about:
* File system mounts
* Overlay file systems (Docker layers)
* pivot_root (similar to chroot but more secure)
* seccomp-bf
* networking
